## Asynchronous Execution

You can define a task to be executed asynchronously. This means that the crew will not wait for it to be completed to continue with the next task. This is useful for tasks that take a long time to be completed, or that are not crucial for the next tasks to be performed.
You can then use the context attribute to define in a future task that it should wait for the output of the asynchronous task to be completed.

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from crewai import Agent,LLM,Crew,Task
from crewai_tools import TavilySearchTool
from crewai.tools import tool

In [3]:
llm = LLM(model="groq/llama-3.3-70b-versatile", temperature=0.2)

In [4]:
search_tool = TavilySearchTool(max_results=2,topic="finance")

In [6]:
market_analysis_agent = Agent(
    role="Senior Stock Market Analyst",
    goal="Perform deep financial and market analysis for the stock symbol {symbol}.",
    backstory="""
    You are a senior equity research analyst with 15+ years of experience.
    You specialize in analyzing company financials, profitability, valuation metrics,
    revenue growth, earnings reports, and overall market trends.
    You provide structured, professional financial insights.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [7]:
future_trend_agent = Agent(
    role="Investment Forecast Strategist",
    goal="Predict short-term future outlook and hype potential of stock {symbol}.",
    backstory="""
    You are a financial forecasting expert skilled in analyzing
    market sentiment, stock momentum, macroeconomic signals,
    institutional activity, and growth catalysts.
    You provide realistic, probability-based forecasts.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [11]:
risk_analysis_agent = Agent(
    role="Financial Risk Assessment Specialist",
    goal="Identify risks, weaknesses, and potential threats for stock {symbol}.",
    backstory="""
    You are a risk auditor who identifies volatility risks,
    regulatory threats, competition challenges, lawsuits,
    declining revenues, and economic pressure.
    You focus on downside protection analysis.
    """,
    tools=[search_tool],
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [12]:
report_aggregator_agent = Agent(
    role="Chief Investment Report Strategist",
    goal="Aggregate all stock analysis and generate final professional report for {symbol}.",
    backstory="""
    You are a senior financial strategist responsible for preparing
    executive-level stock evaluation reports.
    You combine multiple expert analyses into one clear,
    structured and actionable investment report.
    """,
    llm=llm,
    max_tokens=500,
    verbose=True
)


In [13]:
market_analysis_task = Task(
    description="""
    Perform detailed financial and market analysis for stock symbol {symbol}.

    Include:
    - Current market trend
    - Revenue and profit performance
    - Financial metrics (P/E, EPS, Market Cap if available)
    - Recent quarterly performance
    - Overall financial health
    """,
    expected_output="Structured financial analysis report.",
    agent=market_analysis_agent,
    async_execution=True # first 
)


In [14]:
future_trend_task = Task(
    description="""
    Analyze the future outlook of stock {symbol}.

    Include:
    - Short-term prediction (next 2 months)
    - Growth potential
    - Market hype level (High/Moderate/Low)
    - Investment attractiveness
    """,
    expected_output="Future investment outlook report.",
    agent=future_trend_agent,
    async_execution=True # 
)


In [15]:
risk_analysis_task = Task(
    description="""
    Perform risk and downside analysis for stock {symbol}.

    Include:
    - Recent losses
    - Industry competition threats
    - Regulatory risks
    - Market volatility
    - Any current issues
    """,
    expected_output = "Detailed risk analysis report.",
    agent = risk_analysis_agent,
    async_execution = True
)


In [16]:
final_report_task = Task(
    description="""
    Using all previous analyses, generate a comprehensive investment report for {symbol}.

    The report must include:

    1. Executive Summary
    2. Market Performance Overview
    3. Future Outlook (Next 2 Months)
    4. Risk Assessment
    5. Final Recommendation (Buy / Hold / Sell with justification)

    Make it professional and investor-ready.
    """,
    expected_output="Complete stock evaluation report.",
    agent=report_aggregator_agent,
    context=[
        market_analysis_task,
        future_trend_task,
        risk_analysis_task
    ]
)


In [17]:
from crewai import Process

In [18]:
crew = Crew(
    agents=[
        market_analysis_agent,
        future_trend_agent,
        risk_analysis_agent,
        report_aggregator_agent
    ],
    tasks=[
        market_analysis_task,
        future_trend_task,
        risk_analysis_task,
        final_report_task
    ],
    verbose=True
)

In [19]:
result = crew.kickoff(inputs={"symbol": "AAPL"})   # inputs={"symbol": "TSLA"}
print(result)

╭─────────────────────────────────────────── 🚀 Crew Execution Started ───────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name:                                                                                                          │
│  crew                                                                                                           │
│  ID:                                                                                                            │
│  64accb2a-029a-4415-b2ce-e96c1eaf7280                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Perform detailed financial and market analysis for stock symbol AAPL.                                      │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Current market trend                                                                                     │
│      - Revenue and profit performance                                                                           │
│      - Financial metrics (P/E, EPS, Market Cap if available)                                                    │
│      - Recent quarterly performance                                                                             │
│      - Overall financial health                                                                                 │
│                                                                                                                 │
│  ID: f20832eb-3e12-467e-9165-847f6c26b16f                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Analyze the future outlook of stock AAPL.                                                                  │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Short-term prediction (next 2 months)                                                                    │
│      - Growth potential                                                                                         │
│      - Market hype level (High/Moderate/Low)                                                                    │
│      - Investment attractiveness                                                                                │
│                                                                                                                 │
│  ID: d85c41f1-b236-4026-a219-85cb892ad8b9                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Senior Stock Market Analyst                                                                             │
│                                                                                                                 │
│  Task:                                                                                                          │
│      Perform detailed financial and market analysis for stock symbol AAPL.                                      │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Current market trend                                                                                     │
│      - Revenue and profit performance                                                                           │
│      - Financial metrics (P/E, EPS, Market Cap if available)                                                    │
│      - Recent quarterly performance                                                                             │
│      - Overall financial health                                                                                 │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 📋 Task Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Started                                                                                                   │
│  Name:                                                                                                          │
│      Perform risk and downside analysis for stock AAPL.                                                         │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Recent losses                                                                                            │
│      - Industry competition threats                                                                             │
│      - Regulatory risks                                                                                         │
│      - Market volatility                                                                                        │
│      - Any current issues                                                                                       │
│                                                                                                                 │
│  ID: 528b7212-3d02-4b77-bbb2-c28dbd5ff30c                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

[CrewAIEventsBus] Warning: Event pairing mismatch. 'agent_execution_error' closed 'llm_call_started' (expected 
'agent_execution_started')

╭───────────────────────────────────────────────── ❌ LLM Error ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  LLM Call Failed                                                                                                │
│  Error: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for     │
│  model `llama-3.3-70b-versatile` in organization `org_01khtgny8zf6atasq557c9wf4j` service tier `on_demand` on   │
│  tokens per minute (TPM): Limit 12000, Used 10226, Requested 2835. Please try again in 5.305s. Need more        │
│  tokens? Upgrade to Dev Tier today at                                                                           │
│  https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}                      │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

[CrewAIEventsBus] Warning: Event pairing mismatch. 'task_failed' closed 'llm_call_started' (expected 
'task_started')

╭──────────────────────────────────────────────── 📋 Task Failure ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name:                                                                                                          │
│                                                                                                                 │
│      Perform risk and downside analysis for stock AAPL.                                                         │
│                                                                                                                 │
│      Include:                                                                                                   │
│      - Recent losses                                                                                            │
│      - Industry competition threats                                                                             │
│      - Regulatory risks                                                                                         │
│      - Market volatility                                                                                        │
│      - Any current issues                                                                                       │
│                                                                                                                 │
│  Agent:                                                                                                         │
│  Financial Risk Assessment Specialist                                                                           │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.3-70b-versatile` in organization `org_01khtgny8zf6atasq557c9wf4j` service tier `on_demand` on tokens per minute (TPM): Limit 12000, Used 10458, Requested 2557. Please try again in 5.075s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
